In [ ]:
import glob
import json
import os

import librosa
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import soundfile as sf 

import data_processing as dp
import plots

# listen to audios
import IPython.display as ipd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
dataset_path = "/media/gigibs/DD02EEEC68459F17/datasets"
# dataset_path = "/scratch/gv2167/datasets/"
#dataset_path = "/home/gigibs/Documents/datasets"

In [ ]:
qfs = pd.read_json("../data/output_data/qwen_fs.json")
qzs = pd.read_json("../data/output_data/qwen_zs.json")
mfs = pd.read_json("../data/output_data/mu_fs.json")
mzs = pd.read_json("../data/output_data/mu_zs.json")

In [ ]:
qfs = dp.parse_df(qfs, experiment_name="qwen_fs")
qzs = dp.parse_df(qzs, experiment_name="qwen_zs")
mfs = dp.parse_df(mfs, experiment_name="mu_fs")
mzs = dp.parse_df(mzs, experiment_name="mu_zs")

demo data input format:

```json
{
    "title": "example", // maybe we can skip this too
    "text_tokens": [list, of, tokens],
    "text_shapley_values": [list, of, values],
    "sample_rate": 10,
    "total_duration": 10,
    "num_audio_samples": 16000,
    "audio_signal": [list, of, samples],
    "audio_shapley_values": [list, of, values],
    "gt_start": 0.1 (or None),
    "gt_end": 0.2 (or None),
}
```

MAP JSON -> DF
title -> question.id?
question_tokens -> input_tokens
text_shapley_tokens -> **(LOAD)** text shapley 
sample_rate -> **(INFER)** 16k or 24k depending on the model
total_duration -> **(INFER)** from audio samples and sample rate
audio_signal -> **(LOAD)** audio 
audio_shapley_values -> **(LOAD)** audio shapley 
gt_start, gt_end -> **(LOAD)** from sse if exists, or None

CHANGES NEEDED IN THE JSON
text_tokens -> question_tokens

In [ ]:
exp_info = {
    "qwen_fs": {"model": "Qwen-Audio", "experiment": "MCPI"},
    "qwen_zs": {"model": "Qwen-Audio", "experiment": "MCNPI"},
    "mu_fs": {"model": "MU-LLaMA", "experiment": "MCPI"},
    "mu_zs": {"model": "MU-LLaMA", "experiment": "MCNPI"},
}

In [ ]:
def load_shapley_values(row):
    question_id = row.Index
    data = f"../{row.output_folder}/{question_id}_info.npz"
    tokens = row.input_ids
    audio_tokens = np.where(tokens < 0)[-1]
    question_tokens = np.where(tokens >=0)[-1]

    all_shapley_values = data["shapley_values"].squeeze(0).squeeze(0)
    audio_shapley_values = all_shapley_values[audio_tokens]
    question_shapley_values = all_shapley_values[question_tokens]

In [ ]:
def row_to_json(row, dataset_path):
    if row.experiment.startswith("qwen"):
        sample_rate = 16000
    else:
        sample_rate = 24000

    x, fs = librosa.load(os.path.join(dataset_path, row.audio_path), sr=sample_rate)
    #all_shapley_values, audio_shapley_values, question_shapley_values = dp.load_shapley_values(sample)
    output = {}
    output["title"] = row.id
    all_shapley_values, audio_shapley_values, question_shapley_values = dp.load_shapley_values(row)
    
    output["all_shapley_values"] = all_shapley_values.tolist()
    output["audio_shapley_values"] = audio_shapley_values.tolist()
    output["question_shapley_values"] = question_shapley_values.tolist()
    output["question_tokens"] = row.input_tokens.tolist()
    output["answer_tokens"] = row.output_tokens.tolist()
    output["audio_signal"] = x.tolist()
    output["total_duration"] = len(x)/sample_rate
    
    output["num_shapley_samples"] = audio_shapley_values.shape[0]
    output["sample_rate"] = sample_rate
    output["gt_start"] = None
    output["gt_end"] = None

    output["model"] = exp_info[row.experiment]["model"]
    output["experiment"] = exp_info[row.experiment]["experiment"]

    return output

In [ ]:
test = qfs.loc[[427, 719]]

In [ ]:
for i in test.itertuples():
    print(i.Index)
    a = row_to_json(i, dataset_path)

    with open(f"../../2025_investigating_mmshap_demo/data/{i.Index}.json", "w") as f:
        json.dump(a, f)

In [ ]:
with open(f"../../2025_investigating_mmshap_demo/data/{i.Index}.json", "r") as f:
    tmp = json.load(f)

In [ ]:
np.array(tmp["audio_shapley_values"]).shape